In [3]:
import importlib
import sys
import numpy as np
# the mock-0.3.1 dir contains testcase.py, testutils.py & mock.py
sys.path.append('/workspace/src/')
sys.path.append('/workspace/src/deepct')

In [2]:
!pip install nltk
import ir_datasets
dataset = ir_datasets.load('argsme/2020-04-01')

import nltk
nltk.download('stopwords')
nltk.download('punkt')
from tqdm import tqdm
import json
calculate_doc_term_recall = importlib.import_module('deepct.train-deep-ct').calculate_doc_term_recall
calculate_qtokens = importlib.import_module('deepct.train-deep-ct').calculate_qtokens
text_clean = importlib.import_module('deepct.train-deep-ct').text_clean

!nosetests
from deepct.util import clean_qtokens

from nltk.stem.porter import PorterStemmer
stemmer = PorterStemmer()

def calculate_qtokens2(query):
    ret = set()
    qtext = text_clean(query, True, True)
    tokens = set(qtext.split(' '))
    for t in tokens:
        ret.add(t)

    return ret

def calculate_doc_term_recall2(lines, tokens):
    i=1
#     args = recommended_args()
    for json_dict in lines:
        json_dict["term_recall"] = {}

        doc_text = json_dict['doc']['title']
        doc_text = text_clean(doc_text, False, True)# doc_text = text_clean(doc_text, False, args.stop)
        doc_tokens = set(doc_text.split(' '))
        doc_term_recall = {}
        for ttoken in doc_tokens:
            ttoken2 = ttoken
            if True: ttoken2 = stemmer.stem(ttoken) # if args.stem: ttoken2 = stemmer.stem(ttoken)
            if ttoken2 in tokens:
                doc_term_recall[ttoken] = 1.0
        json_dict["term_recall"] = doc_term_recall
        json_dict["doc"]["position"] = 1
        json_dict["doc"]["id"] = i
        i=i+1

        yield json_dict

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

[nltk_data] Downloading package stopwords to /mnt/ceph/storage/data-
[nltk_data]     tmp/2021//angwr/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /mnt/ceph/storage/data-
[nltk_data]     tmp/2021//angwr/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [5]:
path = '/mnt/ceph/storage/data-in-progress/data-teaching/theses/wstud-thesis-hollatz/'
dirname = 'DeepCT_data/'
conclusion_tokens = {}
labeled_arguments_ids = np.load('labeled_arguments.npy')
for doc in tqdm(dataset.docs_iter(), total=387740):
    if doc.doc_id not in labeled_arguments_ids:
        conclusion_tokens[doc.doc_id] = list(calculate_qtokens2(doc.conclusion))

# #replacement to 'cc-2019-47-deepct-qtokens.json'
json.dump(conclusion_tokens, open(path + 'argsme2020-04-01-conclusions-tokens.json', 'w')) 

100%|█████████████████████████████████████████████████████████████████████████| 387740/387740 [01:25<00:00, 4549.54it/s]


In [9]:
with open(path + dirname + 'argsme2020-04-01-passages-without-conclusions-and-leakage.jsonl') as docs, open(path + dirname + 'argsme2020-04-0-train-data-conclusions-no-leakage.jsonl', 'w') as out_file:
    for k, doc in tqdm(enumerate(docs), total=387740):
        doc = json.loads(doc)
        doc_id = doc['id']
        if doc_id not in labeled_arguments_ids:
            if doc_id not in conclusion_tokens:
                continue

            for passage_id, passage in doc['passages']:
                for i in calculate_doc_term_recall2([{"doc":{"title": passage}}], conclusion_tokens[doc_id]):
                    i['doc']['position'] = passage_id
                    i['doc']['id'] = doc_id + '___' + str(passage_id)
                    out_file.write(json.dumps(i) + '\n')

 99%|█████████████████████████████████████████████████████████████████████████▎| 384334/387740 [18:03<00:09, 354.73it/s]
